# Custom property package for CO2, H2O VLE system 

The CO2, H2O mixture is commonly observed as a part of carbon capture systems with water rich solvents, as well as carbon storage systems. In order to simulate and optimize these systems, it is important to understand the thermodynamic properties and equilibrium behavior of this mixture. 

The aim of this tutorial is as follows:
1. Develop a property package for CO2, H2O system phase equilibrium, using the generic property package framework in IDAES, in a stepwise manner.
2. Implement the property package for simulating a flash unit model, and study the VLE behavior 

The assumptions while building this property package is as follows:
1. The vapor and liquid phases show ideal behavior
2. CO2 is a non condensable, and its concentration is negligible in liquid phase
3. The package would be applicable to systems at a pressure close to 1 atm 

Reference Temperature: 298.15 K

Reference Pressure: 101325 Pa

## Part 1: Property Package Development 

### Step 1: Import the required Python and Pyomo packages

1. 'logging' is a python package required to set up the logger
2. 'units' is a Pyomo package required for specifying the units of physical quantities

In [1]:
import logging
from pyomo.environ import units as pyunits

### Step 2: Import IDAES core packages 

IDAES cores required for building the property package need to be imported. They are as follows:

1. 'LiquidPhase', 'VaporPhase', 'PhaseType', are packages that are required to declare phase types for components

2. 'Component' is the package required to declare different system constituents as components in the system

3. 'FTPx' is the package required to specify the system state definition being molar flow rate, temperature, pressure, and mole fractions. 

4. 'Ideal' is the package required to apply ideal equation of state for phases that show ideal behavior.

5. 'smooth_VLE' is the package required to apply smooting function for phase equilibrium formulations, to ensure continuous behavior. 

6. 'IdealBubbleDew' is the package required to apply ideal VLE equations to calculate bubble and dew points for the system

7. 'fugacity' is the package required to apply fugacity based VLE correlations for the system

8. 'Perrys' is the package required to access thermodynamic property corelations from Perry's Chemical Engineer's Handbook (7th Edition), and apply it to the system

9. 'NIST' is the package required to access thermodynamic property corelations from the NIST Web-book, and apply it to the system

In [2]:
from idaes.core import LiquidPhase, VaporPhase, Component
from idaes.core.phases import PhaseType as PT
from idaes.generic_models.properties.core.state_definitions import FTPx
from idaes.generic_models.properties.core.eos.ideal import Ideal
from idaes.generic_models.properties.core.phase_equil import smooth_VLE
from idaes.generic_models.properties.core.phase_equil.bubble_dew import \
        IdealBubbleDew
from idaes.generic_models.properties.core.phase_equil.forms import fugacity
import idaes.generic_models.properties.core.pure.Perrys as Perrys
import idaes.generic_models.properties.core.pure.NIST as NIST

### Step 3: Set up the logger

In [3]:
_log = logging.getLogger(__name__)

### Step 4: Set up the configuration dictionary 

A nested dictionary needs to be created in order to specify the components involved in the system, their thermophysical and thermodynamic property corelations, corresponding parameters, and valid phases. Along with this, it is also meant to provide system level details, like the combination of state variables for state definition, valid bounds for each of them, reference state, and phases in equilibrium. 

The configuration dictionary has two main types of keys:
1. System specific
2. Component specific

The outermost keys are system specific. Theor corresponding values are either specified through a number, methods imported from idaes core, or another dictionary containing system/component specific keys. 

#### Outermost Keys

1. "components" - value: contains another dictionary to specify component details
2. "phases" - value: contains a dictionary of the phases present in the system, along with the equation of states required to define their behavior 
3. "base_units" - value: contains a dictionary of units applicable for different physical quantities and time
4. "state_definition" - value: contains the particular state definition method from idaes core library, according to the desired state variable combinations to be specified for it.
5. "state_bounds" - value: contains a dictionary of the state variable bounds within which the system would operate, and the property package would be valid.
6. "pressure_ref" - value: contains a tuple with the systems's reference pressure value, with the appropriate units
7. "temperature_ref" - value: contains a tuple with the systems's reference temperature value, with the appropriate units
8. "phases_in_equilibrium" - value: contains a list of phases in equilibrium for the system
9. "phase_equilibrium_state" - value: contains a dictionary of the method required from idaes core library to obtain a smooth formulation for phase equilibrium equations
10. "bubble_dew_method" - value: contains the method required from idaes core library to calculate bubble and dew points for the system.